# [TorchServe](https://github.com/pytorch/serve)

In [1]:
!git clone https://github.com/pytorch/serve.git

Cloning into 'serve'...
remote: Enumerating objects: 20318, done.
remote: Counting objects: 100% (1002/1002), done.
remote: Compressing objects: 100% (438/438), done.
remote: Total 20318 (delta 602), reused 884 (delta 539), pack-reused 19316
Receiving objects: 100% (20318/20318), 43.07 MiB | 39.94 MiB/s, done.
Resolving deltas: 100% (11777/11777), done.


## Install TorchServe and torch-model-archiver

In [2]:
%cd serve

/content/serve


In [4]:
# For CPU
#!python ./ts_scripts/install_dependencies.py

# For GPU with Cuda 11.0
#!python ./ts_scripts/install_dependencies.py --cuda=cu110
!python ./ts_scripts/install_dependencies.py --cuda=cu111

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:6 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [60.9 kB]
Ign:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:12 http://security.ubuntu.com/ubu

## Serve a model

### [Serving Huggingface Transformers using TorchServe](https://github.com/pytorch/serve/tree/master/examples/Huggingface_Transformers)


In [5]:
%cd /content/serve/examples/Huggingface_Transformers

/content/serve/examples/Huggingface_Transformers


In [6]:
!mkdir model_store

In [7]:
!pip install transformers

     |████████████████████████████████| 2.2 MB 28.8 MB/s 
     |████████████████████████████████| 3.3 MB 56.2 MB/s 
     |████████████████████████████████| 895 kB 62.5 MB/s 


In [ ]:
#!perl -pi.bak -e "s|,do_lower_case=do_lower_case||g" Download_Transformer_models.py
#!diff Download_Transformer_models.py.bak Download_Transformer_models.py

22c22
<         tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name,do_lower_case=do_lower_case)
---
>         tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name)
26c26
<         tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name,do_lower_case=do_lower_case)
---
>         tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name)
30c30
<         tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name,do_lower_case=do_lower_case)
---
>         tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name)


In [ ]:
#!perl -pi.bak -e "s|, do_lower_case=self.setup_config\[\"do_lower_case\"\]||g;s|do_lower_case=self.setup_config\[\"do_lower_case\"\],\n||g" Transformer_handler_generalized.py
#!diff Transformer_handler_generalized.py.bak Transformer_handler_generalized.py
#!mv Transformer_handler_generalized.py.bak Transformer_handler_generalized.py

75c75
<                 model_dir, do_lower_case=self.setup_config["do_lower_case"]
---
>                 model_dir
80,81c80
<                 do_lower_case=self.setup_config["do_lower_case"],
<             )
---
>                             )


In [ ]:
#!python Download_Transformer_models.py

2021-05-12 05:49:03.592711: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
Transformers version 4.5.1
Download model and tokenizer bert-base-uncased
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you exp

In [9]:
#%cd model_store
%cd ../

/content/serve/examples/Huggingface_Transformers


In [13]:
!pip install "fugashi[unidic-lite]" 
!pip install ipadic

     |████████████████████████████████| 47.4 MB 44 kB/s 
  Created wheel for unidic-lite: filename=unidic_lite-1.0.8-py3-none-any.whl size=47658837 sha256=2afaf971f82bd635d76ec305c5e72a36b1107737c8ee1664bcb60cca0f8a0cb6
  Stored in directory: /root/.cache/pip/wheels/de/69/b1/112140b599f2b13f609d485a99e357ba68df194d2079c5b1a2
Successfully built unidic-lite
     |████████████████████████████████| 13.4 MB 212 kB/s 
  Created wheel for ipadic: filename=ipadic-1.0.0-py3-none-any.whl size=13556723 sha256=c825c8669dac2d74f0a99e46b4102f3eb0753861f48de1a90b4e8fca23dcd9bb
  Stored in directory: /root/.cache/pip/wheels/33/8b/99/cf0d27191876637cd3639a560f93aa982d7855ce826c94348b
Successfully built ipadic


In [17]:
from transformers import AutoModelForSequenceClassification, BertJapaneseTokenizer

# Download a model and a tokenizer.
model = AutoModelForSequenceClassification.from_pretrained('daigo/bert-base-japanese-sentiment') 
tokenizer = BertJapaneseTokenizer.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking')

# Save the model and the tokenizer.
dir_name = 'bert_model'
tokenizer.save_pretrained(dir_name)
model.save_pretrained(dir_name)


In [20]:
!wget https://gist.githubusercontent.com/MFreidank/3463d407a94ffe53d0d0daa137ad3973/raw/6b54f5329e1e7a764e5c9fbff9e8bf0054fe6e4c/transformers_classifier_torchserve_handler.py

--2021-05-25 02:34:33--  https://gist.githubusercontent.com/MFreidank/3463d407a94ffe53d0d0daa137ad3973/raw/6b54f5329e1e7a764e5c9fbff9e8bf0054fe6e4c/transformers_classifier_torchserve_handler.py
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3561 (3.5K) [text/plain]
Saving to: ‘transformers_classifier_torchserve_handler.py’

transformers_classi 100%[===================>]   3.48K  --.-KB/s    in 0s      

2021-05-25 02:34:33 (54.7 MB/s) - ‘transformers_classifier_torchserve_handler.py’ saved [3561/3561]



In [15]:
!pip install torch-model-archiver

In [ ]:
#!torch-model-archiver --model-name BERTSeqClassification --version 1.0 --serialized-file Transformer_model/pytorch_model.bin --handler ./Transformer_handler_generalized.py --extra-files "Transformer_model/config.json,./setup_config.json,./Seq_classification_artifacts/index_to_name.json"

In [21]:
!torch-model-archiver --model-name "bert" --version 1.0 --serialized-file ./bert_model/pytorch_model.bin --extra-files "./bert_model/config.json,./bert_model/vocab.txt" --handler "./transformers_classifier_torchserve_handler.py"

In [22]:
!mkdir -p model_store
#!mv BERTSeqClassification.mar model_store/
!mv bert.mar model_store/

## Start TorchServe to serve the model

In [23]:
!pip install torchserve

     |████████████████████████████████| 18.1 MB 22 kB/s 


### [port in use: try to test/run torchserve in colab (Torchserve stopped. java.io.IOException: Failed to bind to address: http://127.0.0.1:8080) #346](https://github.com/pytorch/serve/issues/346#issuecomment-628411129)

In [24]:
%%writefile config.properties
inference_address=http://127.0.0.1:8880
management_address=http://127.0.0.1:8881
metrics_address=http://127.0.0.1:8882

Writing config.properties


In [35]:
get_ipython().system_raw(
    #'torchserve --start --model-store model_store --models my_tc=BERTSeqClassification.mar --ts-config config.properties &'
    'torchserve --start --model-store model_store --models bert=bert.mar --ts-config config.properties &'
)

In [36]:
!curl http://localhost:8880/ping

{
  "status": "Healthy"
}


In [26]:
%%writefile unhappy_sentiment.txt
私はとっても不幸

Writing unhappy_sentiment.txt


In [27]:
!curl -X POST http://127.0.0.1:8880/predictions/bert -T unhappy_sentiment.txt

1

In [28]:
%%writefile happy_sentiment.txt
私はとっても幸せ

Writing happy_sentiment.txt


In [29]:
!curl -X POST http://127.0.0.1:8880/predictions/bert -T happy_sentiment.txt

0

In [37]:
!curl -X POST http://127.0.0.1:8880/explanations/bert -T unhappy_sentiment.txt

1

In [38]:
!curl -X POST http://127.0.0.1:8880/explanations/bert -T happy_sentiment.txt

0

In [ ]:
#!curl -X POST http://127.0.0.1:8880/predictions/my_tc -T ./Seq_classification_artifacts/sample_text.txt

Accepted

In [ ]:
#!curl -X POST http://127.0.0.1:8880/explanations/my_tc -T ./Seq_classification_artifacts/sample_text.txt

{
  "importances": [
    0.0,
    -0.5859627050756424,
    -0.13570754204744964,
    0.10065775319531475,
    -0.09294911424045345,
    -0.25705342798363534,
    0.1452277118361304,
    0.03582634127020333,
    -0.10992920500263736,
    0.22334276596630362,
    0.29781911854376647,
    -0.1423081185606594,
    -0.24605928122455584,
    -0.5473108360394116,
    0.0
  ],
  "words": [
    "[CLS]",
    "bloomberg",
    "has",
    "decided",
    "to",
    "publish",
    "a",
    "new",
    "report",
    "on",
    "the",
    "global",
    "economy",
    ".",
    "[SEP]"
  ],
  "delta": -0.006339518862528232
}

In [34]:
!torchserve --stop

TorchServe has stopped.
